# Milestone 1: Tackling big data on your laptop

## Authors: Neel Phaterpekar, Arash Shamseddini and Charles Suresh

In [6]:
import requests
import json
import os
from urllib.request import urlretrieve
import zipfile
import glob
import pandas as pd
import numpy as np
import re
from memory_profiler import memory_usage
import pyarrow.dataset as ds
import pyarrow.feather as feather
import pyarrow.parquet as pq
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
import rpy2_arrow.pyarrow_rarrow as pyra

In [7]:
%load_ext rpy2.ipython
%load_ext memory_profiler

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [8]:
%%R
library(dplyr)
library(arrow)

<Br><Br>

## 1. Downloading the data
 
In this section, we access rainfall data using the Figshare API. The following code directly grabs the data from Figshare and unzips the file. 

<br>

### 1.1 Download data from figshare

In [4]:
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [5]:
%%time
%memit
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

peak memory: 318.33 MiB, increment: 0.13 MiB
CPU times: user 84.3 ms, sys: 48.2 ms, total: 132 ms
Wall time: 3.56 s


[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [6]:
%%time
files_to_dl = ["data.zip"] # need only this zip file
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True) # create the folder if not exists
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 4.61 s, sys: 3.79 s, total: 8.4 s
Wall time: 1min 8s


The downloaded data is now saved as 'data.zip'. The next step is to use ZipFile to extract the csv files

### 1.2 Unzipping Files 

In [7]:
%%time
%memit
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

peak memory: 321.38 MiB, increment: 0.00 MiB
CPU times: user 15.8 s, sys: 2.28 s, total: 18.1 s
Wall time: 20.2 s


<br>

### 1.3 Observations


| Contributors | Downloading wall time  | Extracting wall time  | 
|:---:|:-----:|:--------:|
| Neel   |   1min 10s   |  23 s      | 
| Arash   | 10min 12s    | 26.6 s         |  
| Charles   | 59.9s   |    19.9s      |


> Suprisingly Arash's time usage for downloading the data.zip was significantly larger than those for Neel and Charles.

## 2. Combining data CSVs

In this section, we utilize the csv files that were attained from the figsharerainfall zip file. The code below concatenates all csv files into a new 'combined_csv' file. 

There were 6 columns in each csv (with the exception of observed_daily_rainfall_SYD.csv). 
Since observed_daily_rainfall_SYD.csv was missing 4 columns, these columns were created and filled with NAs.

The final data frame contains over 62 million rows.

### 2.1 Using pandas

In [9]:
col_names = pd.read_csv("figsharerainfall/ACCESS-CM2_daily_rainfall_NSW.csv").columns.to_list()

obs_df = pd.read_csv("figsharerainfall/observed_daily_rainfall_SYD.csv")
obs_df.insert(1, "lat_min", np.nan, True)
obs_df.insert(2, "lat_max", np.nan, True)
obs_df.insert(3, "lon_min", np.nan, True)
obs_df.insert(4, "lon_max", np.nan, True)
obs_df = obs_df.reindex(columns = col_names)

obs_df.to_csv("figsharerainfall/observed_daily_rainfall_SYD.csv", index = False)

ValueError: cannot reindex from a duplicate axis

In [11]:
%%time
%memit

files = glob.glob('figsharerainfall/*.csv')

df = pd.concat((pd.read_csv(file, header=0, index_col=0)
                .assign(model=re.search(r'/(.*)_d', file)[1])
                for file in files))

df.to_csv("figsharerainfall/combined_data.csv")

peak memory: 473.93 MiB, increment: 0.00 MiB
CPU times: user 6min 19s, sys: 24.9 s, total: 6min 44s
Wall time: 6min 53s


In [12]:
%%sh
du -sh figsharerainfall/combined_data.csv

5.9G	figsharerainfall/combined_data.csv


### 2.3 Summary table

| Contributors| Machine | Combining data wall time (pandas) | Memory |
|:---:|:-----:|:-----:|:--------:|
| Neel   |MacOS|   6min 51s   |  5.6G      | 
| Arash   |Windows|6min 10s    | 5.7G         |  
| Charles   |MacOS| 5min 32s   |   6.6G       |


### 2.2. Discussing observations

The overall process of combining data was slow. Our group agreed that using pandas seemed to be the easiest method. Additionally some group members could not run the DASK on their machine 

> Both run times and memory usages on different machines within the team are pretty similar

> Time usage: 5-7 min

> Memory usage: 5.5-6.6G

## 3. Load the combined CSV to memory and performing a simple EDA

In this section, the combined csv is loaded to memory. We attempt to reduce the memory usage of this large csv file through different methods (changing data dtype and using DASK). Finally, we explore the data from these new data structures.

In [ ]:
%%time
%%memit
df_float64 = pd.read_csv("figsharerainfall/combined_data.csv")

### 3.1.1 Investigating Memory Reduction Approaches

#### Changing dtype of data:

In [ ]:
df_float32 = df_float64.astype('float32', errors='ignore')

In [ ]:
print(f"Memory usage with float64: {df_float64.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df_float32.memory_usage().sum() / 1e6:.2f} MB")

> Memory usage with float32 (2250.50 MB) is much lower than the memory usage with float64 (3500.78 MB).

####  Using Dask:

In [ ]:
%%time
%%memit
ddf = dd.read_csv('figsharerainfall/combined_data.csv', blocksize=25e6)

In [ ]:
cols = ['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']
ddf[cols] = ddf[cols].astype('float32')

### 3.1.2 Python Simple EDA

#### 3.1.2.1 Value Counts of `model`:

##### Pandas DataFrame:

In [ ]:
%%time
%%memit
print(df_float32[["model"]].value_counts())

##### Dask DataFrame:

In [ ]:
%%time
%%memit
print(ddf["model"].value_counts().compute())

##### Loading in Chunks:

In [ ]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figsharerainfall/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

##### Observations:

|                  | Peak Memory | CPU Time | Wall Time |
|:----------------:|:-----------:|:--------:|:---------:|
| Pandas DataFrame |9312.58 MB  |  5.16s     |   5.16s        |
|  Dask DataFrame  | 8822.06 MB |2min 12s     | 45.8          |
|  Loading in Chunks| 6144.35 MB  |    1min 7s   |   1min 8s    |

When calling `value_counts` function:
- By using Dask DataFrame over Pandas DataFrame, we see that the Peak Memory drops by a considerable amount
- However, the CPU and Wall Times are quite a bit higher when using Dask DataFrame over Pandas DataFrame
- Loading the data in chunks as Pandas DataFrame, has, by far, the lowest Peak Memory usage. However, the wall time is the highest of all the three methods considered here

#### 3.1.2.2 Summary statistics of Rainfall (mm/day) across Australia:

##### Pandas DataFrame `float64`:

In [ ]:
%%time
%%memit
print(df_float64[['rain (mm/day)']].describe())

##### Pandas DataFrame `float32`:

In [ ]:
%%time
%%memit
print(df_float32[['rain (mm/day)']].describe())

##### Dask DataFrame `float32`:

In [ ]:
%%time
%%memit
print(ddf[['rain (mm/day)']].describe().compute())

##### Observations:

|                            | Peak Memory | CPU Time | Wall Time |
|:--------------------------:|:-----------:|:--------:|:---------:|
| Pandas DataFrame `float64` |  7153.32 MB           | 4.98s         |  5.69s         |
| Pandas DataFrame `float32` |  6199.48 MB           |   3.66s       | 4.38s          |
|  Dask DataFrame `float32`  |   6412.95 MB          |   2min 20s      |  44.8s         |

When calling the `describe` function:
- The Peak Memory consumed is drops when using the Pandas DataFrame with `float32` numeric columns over the Pandas DataFrame with `float64` numeric columns
- The CPU and Wall Times also drop when switching from the Pandas DataFrame with `float64` to the Pandas DataFrame with `float32` numeric columns
- Interestingly, when compared with the Pandas DataFrame with `float32` numeric columns, the Dask DataFrame with `float32` numeric columns consumes more Peak Memory, and also has much higher CPU and Wall Times

#### 3.1.2.3 BoxPlot of Rainfall (mm/day) across Australia:

In [ ]:
sns.set_theme(style="whitegrid")
sns.set(rc={"figure.figsize": (15, 7.5)}, font_scale=1.25)

##### Pandas DataFrame `float64`:

In [ ]:
%%time
%%memit
sns.boxplot(x=df_float64['rain (mm/day)'], showfliers=False);
plt.title("BoxPlot of Rainfall (mm/day) across Australia");
plt.xlabel("Rain (mm/day)");

##### Pandas DataFrame `float32`:

In [ ]:
%%time
%%memit
sns.boxplot(x=df_float32['rain (mm/day)'], showfliers=False);
plt.title("BoxPlot of Rainfall (mm/day) across Australia");
plt.xlabel("Rain (mm/day)");

##### Dask DataFrame `float32`:

In [ ]:
%%time
%%memit
sns.boxplot(x=ddf['rain (mm/day)'], showfliers=False);
plt.title("BoxPlot of Rainfall (mm/day) across Australia");
plt.xlabel("Rain (mm/day)");

##### Observations:

|                            | Peak Memory | CPU Time | Wall Time |
|:--------------------------:|:-----------:|:--------:|:---------:|
| Pandas DataFrame `float64` |  5824.61 MB           | 2.98s         |   3.66s        |
| Pandas DataFrame `float32` |  5213.86 MB           |  2.28s        |   2.75s        |
|  Dask DataFrame `float32`  |  6539.55 MB           |  6min 16s        |  2min 3s         |


When creating boxplots using `sns.boxplot`:
- The Peak Memory consumed is lower when using the Pandas DataFrame with `float32` numeric columns when compared with using the Pandas DataFrame with `float64` numeric columns
- The CPU and Wall Times are also lower when using the Pandas DataFrame with `float32` numeric columns when compared with using the Pandas DataFrame with `float64` numeric columns
- Interestingly, the Dask DataFrame with `float32` numeric columns consumes the most amount of Peak Memory and also has much higher CPU and Wall times when compared with the Panadas DataFrames

## 4. R Approaches For Data Transfer

In this section, we create different data structures to transfer the data that was previously in python to R. We use arrow tables, parquet files and feather files to explore different methods. We decided that the parquet file would be the optimal choice of method.

### 4.1.1 Create Data Structures

#### Create table:

This table can then be used to create the other files/data structures

In [ ]:
%%time
%%memit

dataset = ds.dataset("figsharerainfall/combined_data.csv", format="csv")
table = dataset.to_table()

#### Create Arrow Object:

Here we are able to use python to create an arrow table that will then be passed into R. 

In [ ]:
%%time
%%memit
## Here we are loading the arrow dataframe that we have loaded previously
arrow_table = pyra.converter.py2rpy(table)

#### Create Feather File:

Here we are creating a feather file that can then be loaded in R

In [ ]:
%%time
feather.write_feather(table, 'figsharerainfall/combined_data.feather')

In [ ]:
%%sh
du -sh figsharerainfall/combined_data.feather

#### Create Parquet File:

Here we are creating a parquet file that can then be loaded in R

In [ ]:
%%time 
pq.write_table(table, 'figsharerainfall/combined_data.parquet')

In [ ]:
%%sh
du -sh figsharerainfall/combined_data.parquet

|                            | Disk Usage | Wall Time | 
|:--------------------------:|:-----------:|:---------:|
| `combined_data.csv` |  6.6G          |      6min 51s   |
| `combined_data.feather` |  1.0G          |    40.6s    | 
|  `combined_data.parquet`  |  544M        |     47.5s     |

### 4.1.2 Transfer File Structure to R

#### Using Arrow Object:

Using the arrow table created above, we can easily transfer the data between python and R. However, this required us to use `pyra.converter.py2rpy()` in python to create the arrow table (shown above). This is an additional step that seems unnecesary; ideally we could just load the file directly into R instead of having to use both python and R to load in data.

**Our opinion**: This method got the job done and it did not take very long relative to the other methods. However, this additional step of having to use python is inconvenient and can be avoided.


In [ ]:
%%time
%%R -i arrow_table
start_time <- Sys.time()
rview <- head(arrow_table)
summary_df <- glimpse(arrow_table)
model_count <- arrow_table %>% collect() %>% count(model)
end_time <- Sys.time()
print(class(arrow_table))
print(rview)
print(summary_df)
print(model_count)
print(end_time - start_time)

#### Using Feather File:
Here we can directly read in the feather file that was created above 

**Our opinion:** This method was a pretty fast way to load in the data.

In [ ]:
%%time
%%R
### her we are showing how much time it took to read a feather file what we wrote in python

start_time <- Sys.time()
r_table <- arrow::read_feather("figsharerainfall/combined_data.feather")
rview <- head(r_table)
summary_df <- glimpse(r_table)
model_count <- r_table %>% count(model)
end_time <- Sys.time()
print(class(r_table))
print(rview)
print(summary_df)
print(model_count)
print(end_time - start_time)

#### Using Parquet File:

Similarly to the previous section, we directly load a file to R. Here we use a parquet file to load in the data. 

**Our opinion:** This will likely be our method of choice. The memory usage of the parquet is the smallest of the 3 methods and the over speed of this method is was also quite fast. 

In [ ]:
%%time
%%R
start_time <- Sys.time()
r_table <- arrow::read_parquet("figsharerainfall/combined_data.parquet")
view <- head(r_table)
summary_df <- glimpse(r_table)
model_count <- r_table %>% count(model)
end_time <- Sys.time()
print(class(r_table))
print(view)
print(summary_df)
print(model_count)
print(end_time - start_time)


|     Charles                       | End Time - Start Time | CPU Time | Wall Time |
|:--------------------------:|:-----------:|:--------:|:---------:|
| Using Arrow Object |  7.617s          | 20.1s         |   7.79s        |
| Using Feather File |  22.245s           |  44.5s        |   23.1s        |
|  Using Parquet File  |  12.161s         |  20.3s        |  13.4s         |

|     Neel                      | End Time - Start Time | CPU Time | Wall Time |
|:--------------------------:|:-----------:|:--------:|:---------:|
| Using Arrow Object |  34.69s          | 15.2s         |   35.3s        |
| Using Feather File |  74.104s           |  25.1s        |   77s        |
|  Using Parquet File  |  56.478s         |  28.1s        |  59.4s         |

### 4.2 Observations

After comparing these 3 methods:

- We can see that is quite a bit of variability between the 3 methods in terms of time. 

- All three of these methods are much more efficient that using csvs in terms of memory and speed. 

- It should be noted that rpy2-arrow is still fairly new. It is possible that in the future this method of conversion can improve and become more popular.

We will pick parquet file format to transfer data from python to R. The reasons are listed as follows:

- The speed with which the file was created was quite fast.
- The memory that the file takes is significantly less than the feather file and arrow table. 
- The time that it takes to transfer the data to R is reasonable (comparable to the other methods)
- The columnar properties of the parquet file would make it quite easy to use in the future. 

## 5. Issues 

Overall, it was pretty frustrating to have to run the entire notebook several times. Working with big data is a slow process and running individual cells that take multiple minutes at a time can be arduous. However, there really is nothing that can be done about that other than trying to use the most efficient data storage methods (which we have done here).


Neel:
- Neel had issues with maximum memory. Neel's computer actually requested that he force quit some applications because possible memory usage was being exceeded. We were unsure what exactly caused this. Neel ended up force quitting everything and restarting Jupyter. This got rid of the problem 

- Neel had quite a few issues with loading in the arrow table into the R cell magic. On the first day, nothing was happening, even after 20 minutes of waiting for the data to load. Again the solution here was to restart applications and try again on a different day. 

- The regex that used to grab the model names from the individual csv files was not working on my computer, although it was working on Arash's. Also not sure why this happened. 

Charles:
- I mainly faced issues regarding the time need to run certain code chunks: I initially tried to plot a time-series plot for rainfall across Australia. However, even after an hour the code chunk for the time-series plot kept running. So, I had to settle for simple Boxplots
- My computer would randomly freeze for extended periods of time while running certain code chunks. To prevent the computer from freezing, I had to close all other background processes and apps